In [20]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import pandas as pd
import pint
import sys
from typing import List

In [21]:
sys.path.append(str(Path.cwd().parent))

In [22]:
from src.utils import safe_parse_quantity, calculate_thermal_entropy_generation
from comsol_module.comsol_classes import COMSOL_VTU

In [23]:
ROOT = Path().cwd().parent
PARAMETER_SPACE = "03"
ROOT
DATA_TYPE = "Training"
time_step_idx = -1
ureg = pint.get_application_registry()

In [24]:
mapped_root = ROOT / "data" / PARAMETER_SPACE / f"{DATA_TYPE}Mapped"
assert mapped_root.exists()
mapped_folders : List[Path]= [folder for folder in mapped_root.iterdir() if not folder.is_file()]

In [25]:
mapped_dic = {}
for folder in mapped_folders:
    npy_file = folder / "Exports" / f"{DATA_TYPE}_temperatures.npy"
    vti_file = [file for file in folder.rglob("*.vt*")][0]
    mapped_dic[folder.stem] = {"npy": npy_file, "vti" : vti_file, "entropy_num" : [], "entropy_gen" : []}

In [26]:
original_root = ROOT / "data" / PARAMETER_SPACE / f"{DATA_TYPE}Original"
vtu_file = sorted([file for file in original_root.rglob("*.vtu")])
original_dic = {}
original_dic["original"] = {"vtu" : vtu_file, "entropy_num" : [], "entropy_gen" : []}
original_dic["original"]["n_cells"] = []
print(len(original_dic["original"]["vtu"]))

0


## Calculate entropy number

In [27]:
param_folder = ROOT / "data" / PARAMETER_SPACE / "Exports"
assert param_folder.exists()
param_files = sorted([path for path in param_folder.rglob(f"{DATA_TYPE}*.csv")])
print(len(param_files))
param_files


300


[PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_000_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_001_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_002_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_003_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_004_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_005_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_006_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_007_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_008_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_009_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/da

### Mapped Meshes for different sizes

In [28]:
for idx_mesh, (key, values) in enumerate(mapped_dic.items()):
    comsol_data = COMSOL_VTU(values["vti"])
    mapped_dic[key]["n_cells"] = comsol_data.mesh.n_cells
    temperatures = np.load(values["npy"])
    print(temperatures.shape)
    mask = ~(temperatures == 0).all(axis=(1, 2)) # omit indices that are not computed yet
    temperatures = temperatures[mask]
    print(temperatures.shape)
    assert len(temperatures) == len(param_files)
    for (temperature, param_file) in zip(temperatures, param_files):
        param_df = pd.read_csv(param_file, index_col = 0)
        param_df['quantity_pint'] = param_df[param_df.columns[-1]].apply(lambda x : safe_parse_quantity(x))
        lambda_therm = (1 - param_df.loc['host_phi', "quantity_pint"]) * param_df.loc['host_lambda', "quantity_pint"] + \
                            param_df.loc['host_phi', "quantity_pint"] * (4.2 * ureg.watt / (ureg.meter * ureg.kelvin))
        t0      = 0.5 * (param_df.loc["T_h", "quantity_pint"] + param_df.loc["T_c", "quantity_pint"])
        delta_T = (param_df.loc['T_h', "quantity_pint"]  - param_df.loc["T_c", "quantity_pint"])
        s0_total, entropy_number = calculate_thermal_entropy_generation(
            comsol_data.mesh,
            temperature[time_step_idx],
            lambda_therm,
            t0,
            delta_T,
        )
        # L = (comsol_data.mesh.bounds.z_max - comsol_data.mesh.bounds.z_min) * ureg.meter
        # print(f"{idx_snap:03d} : {(lambda_therm * delta_T**2) / (L**2 * t0**2):.5e}") #s0_characteristic =
        mapped_dic[key]["entropy_gen"].append(s0_total.magnitude)
        mapped_dic[key]["entropy_num"].append(entropy_number.magnitude)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/thomassimader/Documents/NIRB/data/03/TrainingMapped/s100_100_100_b0_4000_0_5000_-4000_-0/Exports/Training_temperatures.npy'

### Original COMSOL data

In [ ]:
idxs = []
for file, param_file in zip(original_dic["original"]["vtu"], param_files):
    idx = int(file.stem.split("_")[1])
    assert idx == int(param_file.stem.split("_")[1])
    idxs.append(idx)
    param_df = pd.read_csv(param_file, index_col = 0)
    param_df['quantity_pint'] = param_df[param_df.columns[-1]].apply(lambda x : safe_parse_quantity(x))
    lambda_therm = (1 - param_df.loc['host_phi', "quantity_pint"]) * param_df.loc['host_lambda', "quantity_pint"] + \
                         param_df.loc['host_phi', "quantity_pint"] * (4.2 * ureg.watt / (ureg.meter * ureg.kelvin))
    t0      = 0.5 * (param_df.loc["T_h", "quantity_pint"] + param_df.loc["T_c", "quantity_pint"])
    delta_T = (param_df.loc['T_h', "quantity_pint"]  - param_df.loc["T_c", "quantity_pint"])
    comsol_data = COMSOL_VTU(file)
    assert int(file.stem.split('_')[-1]) == idx
    temp_field = comsol_data.format_field("Temperature", time_step_idx)
    s0_total, entropy_number = calculate_thermal_entropy_generation(
        comsol_data.mesh,
        comsol_data.mesh.point_data[temp_field],
        lambda_therm,
        t0,
        delta_T,
    )
    # L = (comsol_data.mesh.bounds.z_max - comsol_data.mesh.bounds.z_min) * ureg.meter
    # print(f"{idx:03d} : {(lambda_therm * delta_T**2) / (L**2 * t0**2):.5e}") #s0_characteristic =
    original_dic["original"]["entropy_gen"].append(s0_total.magnitude)
    original_dic["original"]["entropy_num"].append(entropy_number.magnitude)
    original_dic["original"]["n_cells"].append(comsol_data.mesh.n_cells)

In [ ]:
# print(len(original_dic["original"]["entropy_gen"]))
# print(len(mapped_dic['s100_100_100_b0_4000_0_5000_-4000_0']["entropy_gen"]))

## Plot

In [ ]:
N_SNAPS = len(original_dic["original"]["vtu"])
colors = px.colors.sample_colorscale("turbo", [n/(N_SNAPS) for n in range(N_SNAPS)])
print(N_SNAPS)

320


In [ ]:
fig = go.Figure()
plot_metric = "entropy_num" #"entropy_num"
squarred_errors = np.zeros((N_SNAPS, len(mapped_folders) + 1)) #! Only if original is also included
for idx_snap in range(N_SNAPS):
    entropies = []
    cell_sizes = []
    mapped_meshes = []


    for key, value in mapped_dic.items():
        entropies.append(value[plot_metric][idx_snap])
        cell_sizes.append(value["n_cells"])
        mapped_meshes.append(key)
        
    sorted_indices = np.argsort(np.array(cell_sizes))
    entropies = np.array(entropies)[sorted_indices]
    cell_sizes = np.array(cell_sizes)[sorted_indices]
    mapped_meshes_sorted = [mapped_meshes[i] for i in sorted_indices]
    
    original_n_cells = original_dic["original"]["n_cells"][idx_snap]
    original_entropy_num = original_dic["original"][plot_metric][idx_snap]
    
    idx_org = np.searchsorted(cell_sizes, original_n_cells)
    cell_sizes = np.insert(cell_sizes, idx_org, original_n_cells)
    entropies = np.insert(entropies, idx_org, original_entropy_num)
    mapped_meshes_sorted.insert(idx_org, "Original Comsol Mesh")
    

    rel_error = (entropies - original_entropy_num) / original_entropy_num
    squarred_errors[idx_snap, :] = (entropies - original_entropy_num)**2
    
    mapped_meshes_sorted = np.array(mapped_meshes_sorted, dtype=object)
    idx_snap_array = np.ones_like(rel_error) * idx_snap
    # Stack into 2D array: shape (n_points, 2)
    custom_data = np.column_stack((mapped_meshes_sorted, rel_error * 100, idx_snap_array))
    

    fig.add_trace(go.Scatter(x=cell_sizes,
                             y=entropies,
                                mode='markers+lines',
                                name=f"{idxs[idx_snap]:03d}",
                                opacity=0.4,#
                                customdata=custom_data,
                                hovertemplate=
                                    'Cells: %{x}<br>'
                                    'Entropy number: %{y}<br>'
                                    'Snap Idx: %{customdata[2]:03d}<br>'
                                    'Mapped: %{customdata[0]}<br>'
                                    'Rel Error: %{customdata[1]:.3f}%<extra></extra>',
                                line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    showlegend=False,        # Hide the legend
    title=f"Entropy generation number vs n_cells - Training Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="n_cells [-]",    # Optional: label for x-axis
    yaxis_title=plot_metric,     # Optional: label for y-axis
    xaxis=dict(
        type='log',                               # Set x-axis to logarithmic scale
        tickformat='.4e',                         # Format x-axis ticks to scientific notation with 2 decimal places
    ),
    yaxis=dict(
        tickformat='.2f',                         # Format y-axis ticks to 2 decimal places (optional)
    )
)

fig.write_image(mapped_root / f"Training_Diff_{plot_metric}.png")
fig.write_html(mapped_root / f"Training_Diff_{plot_metric}.html")
fig.show()

python(27790) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [ ]:
import plotly.express as px
import pandas as pd


df = pd.DataFrame({
    "x": original_dic["original"]["entropy_num"],
    "y": mapped_dic['s100_100_100_b0_4000_0_5000_-4000_0']["entropy_num"]
})


fig = px.scatter(df, x="x", y="y", text = np.arange(len(original_dic["original"]["entropy_num"])))

# Add 1:1 line
fig.add_shape(
    type="line",
    x0=min(df["x"]), y0=min(df["x"]),
    x1=max(df["x"]), y1=max(df["x"]),
    line=dict(color="red", dash="dash"),
    name="1:1 Line"
)

# Customize layout
r2 = np.corrcoef(df.x, df.y)[0, 1]
fig.update_layout(
    title=f'Entropy generation number R2 = {r2 * 100:.2f}%',
    xaxis_title='Entropy number original',
    yaxis_title='Entroy number mapped',
)


fig.write_image(mapped_root / "Training_Diff_Scatter_EntropyGeneration.png")

fig.show()

In [ ]:
org = np.asarray(original_dic["original"]["entropy_num"])
mapped = np.asarray(mapped_dic['s100_100_100_b0_4000_0_5000_-4000_0']["entropy_num"])
mse = np.mean((org - mapped)**2)
print(mse)

8.72180557778584e-06


### Errors

In [ ]:
mses = np.sum(squarred_errors, axis=0)/N_SNAPS
meses_dic = {mesh: mse for (mse, mesh) in zip(mses, mapped_meshes_sorted) }
for key, val in meses_dic.items():
    print(f"{key} - MSE : {val:>10.1e}")

Original Comsol Mesh - MSE :    0.0e+00
s100_100_100_b0_4000_0_5000_-4000_0 - MSE :    8.7e-06
